In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime 


In [21]:
results = pd.read_csv('results')
shootouts = pd.read_csv('shootouts')
teams_regions = pd.read_csv('teams_regions.csv')

results = pd.DataFrame(results)

#dropping the columns we don't need
results = results.drop('tournament', axis = 1)
results = results.drop('city', axis = 1)
results = results.drop('neutral', axis = 1)
results = results.drop('country', axis = 1)

#validating the current date format
try:
    datetime.strptime(results['date'][1], '%Y-%d-%m')
    print("Yes")
except ValueError:
    print("Not the right format")

#changing the date format
years = []

for i in range (len(results)):
    dates = results['date'][i]
    year = datetime.strptime(dates, '%Y-%m-%d').strftime('%Y')
    years.append(year) 

#adding the 'year' column to our data set
results["year"] = years
print(results)

Yes
             date      home_team away_team  home_score  away_score  year
0      1872-11-30       Scotland   England           0           0  1872
1      1873-03-08        England  Scotland           4           2  1873
2      1874-03-07       Scotland   England           2           1  1874
3      1875-03-06        England  Scotland           2           2  1875
4      1876-03-04       Scotland   England           3           0  1876
...           ...            ...       ...         ...         ...   ...
42478  2021-07-25     Costa Rica    Canada           0           2  2021
42479  2021-07-25  United States   Jamaica           1           0  2021
42480  2021-07-29         Mexico    Canada           2           1  2021
42481  2021-07-29  United States     Qatar           1           0  2021
42482  2021-08-01  United States    Mexico           1           0  2021

[42483 rows x 6 columns]


In [22]:

#adding necessary variable 
goal_margin = []
home = []
away = []
for k in range (len(results)):   
    d_k = results['home_score'][k] - results['away_score'][k]
    goal_margin.append(abs(d_k))
    if d_k > 0:
        home.append('1')
        away.append('-1')
    elif d_k < 0:
        home.append('-1')
        away.append('1')
    else:
        home.append('0')
        away.append('0')           
results['home'] = home
results['away'] = away
results['goal_margin'] = goal_margin
print(results)


             date      home_team away_team  home_score  away_score  year home  \
0      1872-11-30       Scotland   England           0           0  1872    0   
1      1873-03-08        England  Scotland           4           2  1873    1   
2      1874-03-07       Scotland   England           2           1  1874    1   
3      1875-03-06        England  Scotland           2           2  1875    0   
4      1876-03-04       Scotland   England           3           0  1876    1   
...           ...            ...       ...         ...         ...   ...  ...   
42478  2021-07-25     Costa Rica    Canada           0           2  2021   -1   
42479  2021-07-25  United States   Jamaica           1           0  2021    1   
42480  2021-07-29         Mexico    Canada           2           1  2021    1   
42481  2021-07-29  United States     Qatar           1           0  2021    1   
42482  2021-08-01  United States    Mexico           1           0  2021    1   

      away  goal_margin  
0

In [23]:
#necessary?
regions = pd.read_csv('teams_regions.csv')
regions = pd.DataFrame(regions)
region_codes = []
for element in regions['Region Code']:
    if element not in region_codes:
        region_codes.append(element)
print(region_codes)

['AF1', 'AF2', 'AF3', 'EU1', 'EU2', 'EU3', 'SA1', 'SA2', 'NA1', 'NA2', 'AS1', 'AS2']


In [24]:
#removing every country we don't need in our analysis
useful = []

for j in range (len(results)):
    if results['home_team'][j] in np.array(regions['Country']) and results['away_team'][j] in np.array(regions['Country']):
        useful.append(j)       
results = results.filter(items = useful, axis = 0)
print(results)


             date   home_team    away_team  home_score  away_score  year home  \
10     1879-01-18     England        Wales           2           1  1879    1   
14     1880-03-15       Wales      England           2           3  1880   -1   
16     1881-02-26     England        Wales           0           1  1881   -1   
22     1882-03-13       Wales      England           5           3  1882    1   
24     1883-02-03     England        Wales           5           0  1883    1   
...           ...         ...          ...         ...         ...   ...  ...   
42475  2021-07-20    Honduras        Qatar           0           2  2021   -1   
42476  2021-07-24      Mexico     Honduras           3           0  2021    1   
42477  2021-07-24       Qatar  El Salvador           3           2  2021    1   
42478  2021-07-25  Costa Rica       Canada           0           2  2021   -1   
42480  2021-07-29      Mexico       Canada           2           1  2021    1   

      away  goal_margin  
1

In [25]:
#replacing every country name with their region's code
nec_results = results 
results = results.drop('home_score', axis = 1)
results = results.drop('away_score', axis = 1)
print(results)

             date   home_team    away_team  year home away  goal_margin
10     1879-01-18     England        Wales  1879    1   -1            1
14     1880-03-15       Wales      England  1880   -1    1            1
16     1881-02-26     England        Wales  1881   -1    1            1
22     1882-03-13       Wales      England  1882    1   -1            2
24     1883-02-03     England        Wales  1883    1   -1            5
...           ...         ...          ...   ...  ...  ...          ...
42475  2021-07-20    Honduras        Qatar  2021   -1    1            2
42476  2021-07-24      Mexico     Honduras  2021    1   -1            3
42477  2021-07-24       Qatar  El Salvador  2021    1   -1            1
42478  2021-07-25  Costa Rica       Canada  2021   -1    1            2
42480  2021-07-29      Mexico       Canada  2021    1   -1            1

[8765 rows x 7 columns]


In [26]:
check_countries = np.array(nec_results['away_team'])

In [27]:
for country in results['home_team']:
    a = regions.index[(regions['Country'] == country)].tolist()
    results = results.replace({country : regions['Region Code'][a[0]]})

print(results)

             date home_team away_team  year home away  goal_margin
10     1879-01-18       EU1       EU3  1879    1   -1            1
14     1880-03-15       EU3       EU1  1880   -1    1            1
16     1881-02-26       EU1       EU3  1881   -1    1            1
22     1882-03-13       EU3       EU1  1882    1   -1            2
24     1883-02-03       EU1       EU3  1883    1   -1            5
...           ...       ...       ...   ...  ...  ...          ...
42475  2021-07-20       NA2       AS1  2021   -1    1            2
42476  2021-07-24       NA1       NA2  2021    1   -1            3
42477  2021-07-24       AS1       NA2  2021    1   -1            1
42478  2021-07-25       NA1       NA1  2021   -1    1            2
42480  2021-07-29       NA1       NA1  2021    1   -1            1

[8765 rows x 7 columns]


In [28]:
results.to_excel("Cleaned_results.xlsx", index = False)
nec_results.to_excel('Changed_results.xlsx', index = False)

In [29]:
for it in results['away_team']:
    if it in check_countries:
        print(it)
    else:
        print("No mistake, you bitch")

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

...

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake, you bitch
No mistake

In [ ]:
#Note that cleaned_results will be the import file to work with
#The file "changed_results" is the complete version (with details such as country names) of the file "cleaned_results"